## Load necessary python modules

In [11]:
import os
import napari
import tifffile
import numpy as np
import pandas as pd
import trackpy as tp
import matplotlib.pyplot as plt

import bigfish
import bigfish.plot as plot
import bigfish.stack as stack
import bigfish.detection as detection
import bigfish.multistack as multistack

import tkinter as tk
from tkinter import filedialog

from copy import deepcopy
import dask.array as da
from get3DBlurredSequence import *
from dask.array.image import imread as imr
from bigfish.detection.utils import get_object_radius_pixel
from buildReferenceSpot import buildReferenceSpotFromImages
from runBigfishDetection import getSpotAndClusters, saveSpotsNPZ

pd.set_option('display.max_rows', 1500)

from skimage.measure import label, regionprops
import bigfish.segmentation as segmentation
from skimage.segmentation import find_boundaries


from skimage.morphology.selem import disk
from tkinter import *
from pathlib import Path

## Define functions

In [12]:

def choose_home_folder():
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    file_path = filedialog.askdirectory(initialdir= "/", title='Please select a directory')  # Open file dialog

    root.destroy()  # Close the tkinter window
    return file_path


def findFitstTxFrame(blurClusters):
    for i in range(len(blurClusters)):  
        if blurClusters[i].size!=0:
            return i

def findSpotBrightness2D(spotList, blurImage):
    meanBrightness = []
    for hh in range(len(spotList)):
#         z = spotList[hh,0]
        y = spotList[hh,0]
        x = spotList[hh,1]
        meanBrightness.append(np.sum(np.array(blurImage[y-3:y+4,x-3:x+4])))
    return meanBrightness

from scipy.optimize import curve_fit

def bi_exp(x, a, b, c, d):
    return (a * np.exp(-b * x)) + (c * np.exp(-d * x))

def trip_exp(x, a, b, c, d, e, f):
    return ((a * np.exp(-b * x)) + (c * np.exp(-d * x)) + (e * np.exp(-f * x)))

def getBleachCorrected(stackCell, model='bi'):
    axes = tuple([i for i in range(len(stackCell.shape))])
    I_mean = np.mean(stackCell, axis=axes[1:])
    timePoints = np.arange(stackCell.shape[0])
    
    if model=='bi':
        coeffsExp, _ = curve_fit(bi_exp, timePoints, I_mean, maxfev=50000)
        f_ = np.vectorize(bi_exp)(timePoints, *coeffsExp)
    elif model=='tri':
        coeffsExp, _ = curve_fit(trip_exp, timePoints, I_mean, maxfev=50000)
        f_ = np.vectorize(trip_exp)(timePoints, *coeffsExp)
    
    
    f = f_ / np.max(f_)
    f = f.reshape(-1, 1, 1, 1)
    imagesCorrected = (stackCell / f).astype(np.uint16)

    # calculate r squared
    residuals = I_mean - f_
    ss_res = np.sum(residuals ** 2)
    ss_tot = np.sum((I_mean - np.mean(I_mean)) ** 2)
    r_squared = 1 - (ss_res / ss_tot)
    r_squared_exp = np.array(r_squared)
    return imagesCorrected, r_squared_exp, I_mean

In [13]:

midentifier = 'cell_'
identifier = 'basal'

## Obtain 3D blurred image sequence

In [14]:
baseFolder0 = choose_home_folder()
print("Chosen home folder:", baseFolder0) # folder containg folder of movies
baseFolders = [os.path.join(baseFolder0,i) for i in os.listdir(baseFolder0) if identifier in i and os.path.isdir(os.path.join(baseFolder0,i))]

Chosen home folder: C:/Users/uid-1204/Desktop/test2_bigFISHLIVE


In [15]:
baseFolders.sort()
baseFolders

['C:/Users/uid-1204/Desktop/test2_bigFISHLIVE\\Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06']

In [16]:
for baseFolder in baseFolders[:]:
    sessionNames = [os.path.join(baseFolder, i) for i in os.listdir(baseFolder) if midentifier in i and os.path.isdir(os.path.join(baseFolder,i))]
    try:
        os.makedirs(os.path.join(baseFolder,'blurs'))
        print('Blur folder created')
    except FileExistsError:
       # directory already exists
       pass

    for sessionName in sessionNames[:]:
        print(sessionName)
        imsQ = str(Path(sessionName).name).split('_F')[-1]
        print(imsQ)
        pathToTimeSequenceFullView = os.path.join(sessionName,'*.tif')

        sequenceCell = imr(pathToTimeSequenceFullView)
        mipSequenceCell = np.max(sequenceCell, axis=1)
        blurWindow = 3
        try:
            blurImage = get3DBlurredSequence(sequenceCell, blurWindow)
            blurReshaped = np.asarray(blurImage)
            finalImage = da.from_array(blurReshaped, chunks=blurReshaped.shape)
            pathToSaveFrames = baseFolder+'/blurs/blur_'+imsQ
            extension = '.tif'
            with tifffile.TiffWriter(pathToSaveFrames+extension, imagej=True) as tif:
                tif.write(finalImage)
        except ValueError:
            pass

Blur folder created
C:/Users/uid-1204/Desktop/test2_bigFISHLIVE\Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06\cell_1
cell_1
C:/Users/uid-1204/Desktop/test2_bigFISHLIVE\Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06\cell_10
cell_10
C:/Users/uid-1204/Desktop/test2_bigFISHLIVE\Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06\cell_11
cell_11
C:/Users/uid-1204/Desktop/test2_bigFISHLIVE\Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06\cell_12
cell_12
C:/Users/uid-1204/Desktop/test2_bigFISHLIVE\Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06\cell_13
cell_13
C:/Users/uid-1204/Desktop/test2_bigFISHLIVE\Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06\cell_14
cell_14
C:/Users/uid-1204/Desktop/test2_bigFISHLIVE\Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06\cell_15
cell_15
C:/Users/uid-1204/Desktop/test2_bigFISHLIVE\Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06\cell_16
cell_16
C:/Users/uid-1204/Desktop/test2_bigFISHLIVE\Hela_h9_h2_k11_mcpsg_1hrba

C:\Users\uid-1204\AppData\Local\anaconda3\envs\bigfishLive\lib\site-packages\numpy\core\_methods.py:164: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(a)


C:/Users/uid-1204/Desktop/test2_bigFISHLIVE\Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06\cell_18
cell_18


C:\Users\uid-1204\AppData\Local\anaconda3\envs\bigfishLive\lib\site-packages\numpy\core\_methods.py:164: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(a)


C:/Users/uid-1204/Desktop/test2_bigFISHLIVE\Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06\cell_2
cell_2
C:/Users/uid-1204/Desktop/test2_bigFISHLIVE\Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06\cell_3
cell_3
C:/Users/uid-1204/Desktop/test2_bigFISHLIVE\Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06\cell_4
cell_4
C:/Users/uid-1204/Desktop/test2_bigFISHLIVE\Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06\cell_5
cell_5
C:/Users/uid-1204/Desktop/test2_bigFISHLIVE\Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06\cell_6
cell_6
C:/Users/uid-1204/Desktop/test2_bigFISHLIVE\Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06\cell_7
cell_7
C:/Users/uid-1204/Desktop/test2_bigFISHLIVE\Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06\cell_8
cell_8
C:/Users/uid-1204/Desktop/test2_bigFISHLIVE\Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06\cell_9
cell_9
